# Miguel Angel Ruiz Ortiz
## Procesamiento de Lenguaje Natural
## Tarea 4: Modelo de Lenguaje de Política

In [142]:
from typing import Union, Optional
from pathlib import Path
from itertools import permutations
import re
import nltk
from nltk.tokenize import TweetTokenizer
import math
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Descarga de "Punkt sentence tokenizer" para poder dividir un texto en oraciones.

In [3]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## 2- Textos de las Mañeras

Tokeniza con alguna librería de tu elección para obtener oraciones y crear un corpus de miles de oraciones de todos los textos de las mañaneras. Construye un archivo de texto como el de los tuits, una oración por línea.

---

In [4]:
base_path = Path("/content/drive/My Drive/Academic Stuff/NLP (CIMAT)/corpus")
amlo_conferences_path = base_path / "conferencias-presidencia-mx" / "conferencias-amlo"
sheinbaum_conferences_path = base_path / "conferencias-presidencia-mx" / "conferencias-sheinbaum"
all_corpus_path = base_path / "mx_politics_corpus" / "corpus.txt"

In [5]:
def get_file_paths(dir_path: Path) -> list[str]:
    """
      Get file paths from a given directory
    """
    return list(dir_path.glob("*.txt"))

def get_corpus(file_paths: list[Path]) -> dict[str, str]:
    """
    Get corpus from a list of file paths, with .txt files, as a dictionary { file name -> file content }
    """
    corpus = {}

    for file in file_paths:
      with open(file, "r", encoding="utf-8") as f:
        filename, _ = file.name.split(".") # we quit extension .txt from file name
        text = f.read().replace("\xa0", " ") # \xa0 is a "non-breaking space" found through the text
        corpus[filename] = text

    return corpus

In [6]:
amlo_filenames = get_file_paths(amlo_conferences_path)
shein_filenames = get_file_paths(sheinbaum_conferences_path)

amlo_corpus = get_corpus(amlo_filenames)
shein_corpus = get_corpus(shein_filenames)

In [7]:
print(amlo_corpus["2018-12-07"][:300])

07.12.18 Versión estenográfica de la conferencia de prensa matutina del presidente de México, Andrés Manuel López Obrador

Descarga audio:  07-12-18 AUDIO CONFERENCIA DE PRENSA PRESIDENTE DE MÉXICO
PALACIO NACIONAL, 07 de diciembre de 2018
Versión estenográfica
 
Conferencia de prensa del presidente


In [8]:
corpus = []

for key in amlo_corpus:
    corpus.append(amlo_corpus[key])

for key in shein_corpus:
    corpus.append(shein_corpus[key])

In [9]:
corpus[10]

'25.02.21 Versión estenográfica de la conferencia de prensa matutina del presidente Andrés Manuel López Obrador\n\n2021: Año de la Independencia\n \nPRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Buenos días. Se nos hizo un poco tarde, pero ya estamos aquí.\nSólo tengo un tema que tratarles, de modo que vamos a tener tiempo de contestar preguntas de ustedes, informar a la gente, como siempre lo hacemos. Voy nada más a proceder a desahogar mi tema y luego ya nos vamos con las preguntas y las respuestas.\nMiren, la Auditoría Superior de la Federación dio a conocer un informe y se aprovecharon nuestros adversarios, la prensa conservadora que defiende al régimen corrupto, para afectarnos en la imagen del gobierno, sobre todo en el caso de la cancelación del aeropuerto de Texcoco.\nEntonces, no creo, aunque existe la posibilidad de que hayan hecho mal las cuentas, aun así, sería lamentable que la Auditoría de la Federación hiciera mal las cuentas; más bien, creo que se trata de una actitud politiq

A continuación creamos un archivo con todas las oraciones que se encuentran en las conferencias de Andrés Manuel López Obrador y de Claudia Sheinbaum. Para esto hay que tener cuidado, pues no sólo podemos hacer un ``split`` del texto utilizando los puntos ("."). Hay veces que un punto no determina el final de una oración, si no más bien una abreviación. Para lograrlo, la función ``sent_tokenize`` de NLTK (ver [documentación](https://www.nltk.org/api/nltk.tokenize.html#nltk.tokenize.sent_tokenize)) que utiliza el tokenizador "Punkt Sentence" que descargamos al inicio del notebook.

In [ ]:
def create_sentence_corpus(texts: list[str], file_path: Path) -> None:
    all_sentences = []
    for txt in texts:
        all_sentences += nltk.sent_tokenize(txt, language="spanish")

    with open(file_path, "w", encoding="utf-8") as file:
        file.write("\n".join(all_sentences))

In [11]:
create_sentence_corpus(corpus, all_corpus_path)

In [12]:
with open(all_corpus_path, "r") as corpus_file:
    corpus_sent = [line.replace("\n", "") for line in corpus_file if len(line) > 3] # omit too short lines, such as single character "\n" lines

In [13]:
corpus_sent[:20]

['14.07.21 Versión estenográfica de la conferencia de prensa matutina del presidente Andrés Manuel López Obrador',
 '2021: Año de la Independencia',
 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Buenos días.',
 'Pues como lo hacemos los miércoles, ya se ha venido acreditando poco a poco la sección de Quién es quién en las mentiras de la semana.',
 'Elizabeth nos va a informar sobre esto.',
 'No sobra decir que se hace esta acción, se lleva a cabo esta acción porque es mucho el bombardeo de mentiras en los medios de información, de muchas noticias falsas; y es público y notorio que la mayoría de los medios de información están en una abierta campaña en contra de nosotros, de modo que hay que ir informando, ejerciendo el derecho de réplica y garantizando también el derecho a la libre manifestación de las ideas.',
 'No caer en la censura, en la mordaza, sino que haya libertades para todos.',
 'Los medios pueden ejercer todas sus libertades, todas, todas, todas.',
 'Prohibido prohibir, nada má

Luego tokenizamos cada oración con ``TweetTokenizer`` de NLTK.

In [14]:
tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True)
corpus_sent_tk = [tokenizer.tokenize(text) for text in corpus_sent]

In [15]:
corpus_sent_tk[:2]

[['14.07',
  '.',
  '21',
  'versión',
  'estenográfica',
  'de',
  'la',
  'conferencia',
  'de',
  'prensa',
  'matutina',
  'del',
  'presidente',
  'andrés',
  'manuel',
  'lópez',
  'obrador'],
 ['2021', ':', 'año', 'de', 'la', 'independencia']]

---

## 3- Modelo de Lenguaje y Evaluación

### 3.1- Preprocese todos los textos

 Preprocese todos los textos según su intuición para construir un buen corpus para un modelo de lenguaje (e.g., solo palabras en minúscula, "dejé" o "quité" puntuación, etc.). Agregue tokens especiales de ``<s>`` y ``</s>`` según usted considere (e.g., al inicio y final de cada oración); defina su vocabulario y enmascare con ``<unk>`` toda palabra que no esté en su vocabulario.

---

Contamos el número de tokens en el corpus

In [16]:
all_words = set()

for doc in corpus_sent_tk:
    all_words.update(doc)

In [17]:
len(all_words)

99825

A continuación vamos a procesar los textos. Se va a quitar los signos de puntuación y considerar un vocabulario de los 50,000 tokens más frecuentes. También se va a reemplazar los tokens fuera del vocabulario con el token especial `<unk>`, y se agregarán tokens de inicio y fin de oración, `<s>` y `</s>`. La razón de por qué quitar los signos de puntuación es que sólo vamos a considerar modelos de lenguaje basados en $n$-gramas para $n=1, 2, 3, 4$, y no creo que el modelo sea capaz de predecir dónde poner los signos de puntuación con un contexto tan corto. Los signos de puntuación, desde mi punto de vista, requiere considerar contextos más grandes de al menos el tamaño de una oración promedio. Por ejemplo, para saber dónde va un punto (".") el modelo tiene que ser capaz de saber dónde una oración trasmitió una idea completa y poder separar la siguiente idea con un punto. Para transmitir una idea es usual usar más de 4 palabras.

Esta lógica de preprocesamiento la vamos a incluir en la siguiente clase ``TextProcessor``. Esta recibe el número de tokens más frecuentes en un corpus dado (50,000 en nuestro caso).

In [18]:
class TextProcessor:

    def __init__(self, max_vocab_size: int):
        self.max_vocab_size = max_vocab_size

        # special tokens
        self.INIT_TKN = "<s>"
        self.END_TKN = "</s>"
        self.UNK_TKN = "<unk>"

        # punctuation signs that will be not considered
        self.punctuation = {"¡", "!", '"', "$", "%", "&", "'", "(", ")", "*", "+", ",", "-", ".", ":", ";", "¿", "?", "@", "[", "]", "_", "`", "{", "}", "«", "»", "…"}

        # to be computed in fit method
        self.vocab = { self.UNK_TKN, self.INIT_TKN, self.END_TKN }
        self.vocab_len = 0
        self.word2id = { self.UNK_TKN: max_vocab_size, self.INIT_TKN: max_vocab_size + 1, self.END_TKN: max_vocab_size + 2 } # mapping {word -> word id}
        self.id2word = { max_vocab_size: self.UNK_TKN, max_vocab_size + 1: self.INIT_TKN, max_vocab_size + 2: self.END_TKN } # mapping {word id -> word}

    def mask_oov(self, text: Union[str, list[str]]) -> Union[str, list[str]]:
        """Replace out-of-vocabulary words with <unk>"""
        if isinstance(text, str):
            return text if text in self.vocab else self.UNK_TKN
        else:
            # than it is list of strings
            return list(map(self.mask_oov, text))

    def mask_text(self, text: list[str]) -> list[str]:
        """ Mask if out of vocabulary and add initial and end sentence
        """
        return [self.INIT_TKN] + self.mask_oov(text) + [self.END_TKN]

    def transform(self, corpus: list[list[str]]) -> list[list[str]]:
        return [self.mask_text(text) for text in corpus]

    def fit(self, corpus: list[list[str]]) -> list[list[str]]:
        """Compute vocabulary and mask input corpus
        """
        corpus_words = []
        for doc in corpus:
            corpus_words += doc

        freq_dist = nltk.FreqDist(corpus_words)
        freq_dist_order = freq_dist.most_common()

        idx = 0

        for word, _ in freq_dist_order:
            if word not in self.punctuation:
                self.word2id[word] = idx
                self.id2word[idx] = word

                idx += 1

                if idx >= self.max_vocab_size:
                    break

        self.vocab.update(self.word2id.keys())
        self.vocab_len = len(self.vocab)

        return self.transform(corpus)

In [19]:
processor = TextProcessor(max_vocab_size = 50000)
corpus_masked = processor.fit(corpus_sent_tk)

In [20]:
corpus_masked[10]

['<s>',
 'vendían',
 'medicinas',
 '<unk>',
 'se',
 'dedicaban',
 'a',
 'construir',
 'carreteras',
 '<unk>',
 'hospitales',
 '<unk>',
 'reclusorios',
 '<unk>',
 'obtenían',
 'créditos',
 'de',
 'la',
 'banca',
 'de',
 'desarrollo',
 '<unk>',
 '<unk>',
 'el',
 'presupuesto',
 'y',
 'ahora',
 'todo',
 'eso',
 'se',
 'destina',
 'a',
 'atender',
 'a',
 'la',
 'gente',
 '<unk>',
 'al',
 'pueblo',
 'y',
 'en',
 'especial',
 'a',
 'los',
 'más',
 'pobres',
 '<unk>',
 '</s>']

Confirmamos que el vocabulario tiene los 50,000 palabras más frecuentes del corpus y también los tres tokens especiales:

In [21]:
len(processor.vocab)

50003

---

### 3.2- Entrene cuatro modelos de lenguaje

Entrene cuatro modelos de lenguaje sobre todos los tuits: $P_{\text{unigramas}}(w_1^n)$, $P_{\text{bigramas}}(w_1^n)$, $P_{\text{trigramas}}(w_1^n)$ , $P_{\text{tetragramas}}(w_1^n)$. Para cada uno proporcione una interfaz (función) sencilla para $P_{n\text{-grama}}(w_1^n)$ y $P_{n\text{-grama}}(w_n | w_{n-N+1}^{n-1})$. Los modelos deben tener una estrategia común para lidiar con secuencias de tokens no vistos. Muestre un par de ejemplos de como funciona, al menos uno con una palabra fuera del vocabulario.

---

En la siguiente clase se define la lógica para un modelo de lenguaje basado en $n$-gramas, utilizando la estrategia Add-k para tratar con secuencias de tokens no vistas. Contiene los siguientes métodos:

1.   ``fit``: calcula las frecuencias de los $n$-gramas y $n-1$ gramas en un corpus de training dado.
2.   ``conditional_prob``: calcula la probabilidad condicional de un token dado un contexto de $n-1$ tokens.
3.   ``prob_sequence``: calcula la probabilidad de una secuencia de tokens utilizando la regla de la cadena para la probabilidad conjunta. Puede recibir un parámetro booleano, ``log_prob``, que indica si se debe calcular la log-probabilidad (útil para secuencias largas con probabilidad casi cero).



In [22]:
class NgramLanguageModel:
    # n-gram language model using Add-k strategy (default k=1, which is Laplace/Add-One strategy)

    def __init__(self, n: int, text_processor: TextProcessor, k: float = 1.0):

        self.n = n
        self.k = k
        self.text_processor = text_processor
        self.vocab_len = text_processor.vocab_len

        # to be computed in fit method
        self.corpus_len = 0
        self.n_gram_freq = {}  # frequency of n-grams
        self.n_1_gram_freq = {}  # frequency of (n-1)-grams

    def fit(self, corpus: list[list[str]], mask: bool = False):
        if mask:
            corpus = [self.text_processor.mask_text(text) for text in corpus]

        self.corpus_len = sum(len(text) for text in corpus)

        # we assume only one start-of-sentence token in each sentence
        if self.n >= 3:
            corpus = [[self.text_processor.INIT_TKN]*(self.n-2) + text for text in corpus]

        if self.n > 1:
            # consider the case to compute the probability of a token to be at the beggining of a sentence
            self.n_1_gram_freq[(self.text_processor.INIT_TKN,)*(self.n-1)] = self.corpus_len

        # compute frequency of n-grams in corpus
        for doc in corpus:
            for i in range(self.n-1, len(doc)):
                n_gram = tuple(doc[i-self.n+1 : i+1])  # n-gram
                self.n_gram_freq[n_gram] = self.n_gram_freq.get(n_gram, 0) + 1

                if self.n > 1:
                  n_1_gram = n_gram[1:]  # first (n-1)-gram
                  self.n_1_gram_freq[n_1_gram] = self.n_1_gram_freq.get(n_1_gram, 0) + 1

        return

    def conditional_prob(self, word: str, context: list[str]) -> float:
        if len(context) != self.n - 1:
            raise ValueError("context should be of length n-1")

        word = self.text_processor.mask_oov(word)
        context = self.text_processor.mask_oov(context)

        numerator = self.n_gram_freq.get(tuple(context+[word]), 0) + self.k

        if self.n > 1:
            denominator = self.n_1_gram_freq.get(tuple(context), 0) + self.k * self.vocab_len
        else:
            # unigrams model
            denominator = self.corpus_len + self.k * self.vocab_len

        return numerator / denominator

    def prob_sequence(self, sequence: list[str], log_prob: bool = False) -> float:
        # check if sequence has <s> token at the beggining, and if so add the necessary <s> tokens to compute correctly the probability
        if self.n >= 3 and sequence[0] == self.text_processor.INIT_TKN:
            sequence = [self.text_processor.INIT_TKN]*(self.n-2) + sequence

        if len(sequence) < self.n:
            raise ValueError("sequence should be at least of length n, except in the case where sequence starts with <s> (only one). In the latter case, sequence should be at least of length 2")

        sequence = list(map(self.text_processor.mask_oov, sequence))

        # compute the chain rule for the probability of the sequence (using logarithm)
        log_p = 0
        for i in range(self.n-1, len(sequence)):
            # when len(doc) - n + 1 <= 0, it does not enter to the for loop. This makes sense because that means doc does not have n-grams
            n_gram = sequence[i-self.n+1 : i+1]  # n-gram
            log_p += math.log(self.conditional_prob(n_gram[-1], n_gram[:-1]))

        if log_prob:
            return log_p

        return math.exp(log_p)

Se utilizó un coeficiente de $k=0.1$ para la estrategia. Se optó por este número al ver cuánto bajaba $\mathbb{P}$["matutina'' | "prensa"] con diferentes números $k$ y optando por el que le seguía asignando una probabilidad significativa (e.g., 1/4 de la probabilidad sin suavizado). Dado que "conferencia de prensa matutina" era una oración muy presente en el corpus, quería escoger un $k$ que reflejara una probabilidad $\mathbb{P}$["matutina'' | "prensa"] alta. Este probabilidad se muestra de ejemplo en el modelo de bigramas.

<h4>Ejemplos de modelo de unigramas</h4>


In [23]:
unigram_model = NgramLanguageModel(n=1, text_processor=processor, k=0.1)
unigram_model.fit(corpus_masked)

Probabilidad de token "mafia"

In [51]:
unigram_model.conditional_prob("mafia", [])

1.5135359910603456e-05

Probabilidad de secuencia: "mafia", "del", "poder"

In [25]:
unigram_model.prob_sequence(["mafia", "del", "poder"])

8.374929453164593e-11

Verificación de que la suma de las probabilidades de cada token es cercano a 1.

In [26]:
prob = 0
for w in processor.vocab:
  prob += unigram_model.conditional_prob(w, [])
print(prob)

0.9999999999998704


<h4>Ejemplos de modelo de bigramas</h4>

In [27]:
bigram_model = NgramLanguageModel(n=2, text_processor=processor, k=0.1)
bigram_model.fit(corpus_masked)

Probabilidad de token "matutina", dado el token "prensa"

In [28]:
bigram_model.conditional_prob("matutina", ["prensa"])

0.17794503601447517

Para comparar el efecto del suavizado con k=0.1, aquí se muestre la probabilidad de token "matutina", dado el token "prensa" pero SIN SUAVIZADO.

In [29]:
(bigram_model.n_gram_freq[("prensa", "matutina")])/(bigram_model.n_1_gram_freq[("prensa",)])

0.4217593861331872

Probabilidad de secuencia: "presidente", "de", "estados", "unidos"

In [30]:
bigram_model.prob_sequence(["presidente", "de", "estados", "unidos"])

0.00010493134793891028

Verificación de que la suma de las probabilidades de cada token, dado un contexto fijo, es cercano a 1.

In [31]:
prob = 0
for w in processor.vocab:
    prob += bigram_model.conditional_prob(w, ["los"])
print(prob)

1.000000000000698


<h4>Ejemplos de modelo de trigramas</h4>

In [32]:
trigram_model = NgramLanguageModel(n=3, text_processor=processor, k=0.1)
trigram_model.fit(corpus_masked)

Probabilidad de token "buenos", dado el contexto ["``<s>``", "``<s>``"]

In [33]:
trigram_model.conditional_prob("buenos", ["<s>", "<s>"])

0.00010763186939256861

Probabilidad de secuencia: "la", "educación", "es"

In [34]:
trigram_model.prob_sequence(["la", "educación", "es"])

0.009632693443997089

Verificación de que la suma de las probabilidades de cada token, dado un contexto fijo, es cercano a 1.

In [35]:
prob = 0
for w in processor.vocab:
    prob += trigram_model.conditional_prob(w, ["de", "los"])
print(prob)

0.9999999999999263


<h4>Ejemplos de modelo de tetragramas</h4>

In [36]:
tetragram_model = NgramLanguageModel(n=4, text_processor=processor, k=0.1)
tetragram_model.fit(corpus_masked)

Probabilidad de token "matutina", dado el contexto ["conferencia", "de", "prensa"],

In [37]:
tetragram_model.conditional_prob("matutina", ["conferencia", "de", "prensa"])

0.21374445845077403

Probabilidad de secuencia: "``<s>``", "buenos", "días"

In [38]:
tetragram_model.prob_sequence(["<s>", "buenos", "días"])

3.315523388156853e-05

Verificación de que la suma de las probabilidades de cada token, dado un contexto fijo, es cercano a 1.

In [39]:
prob = 0
for w in processor.vocab:
    prob += tetragram_model.conditional_prob(w, ["la", "mafia", "del"])
print(prob)

0.99999999999982


Probabilidad de token no visto en cada uno de los modelos con un contexto dado.

In [40]:
oov_word = "capibara"
print(oov_word in processor.vocab)

False


In [41]:
print("Modelo de unigramas:", unigram_model.conditional_prob(oov_word, []))
print("Modelo de bigramas:", bigram_model.conditional_prob(oov_word, ["el"]))
print("Modelo de trigramas:", trigram_model.conditional_prob(oov_word, ["mafia", "del"]))
print("Modelo de tetragramas:", tetragram_model.conditional_prob(oov_word, ["conferencia", "de", "prensa"]))

Modelo de unigramas: 0.13086048831885896
Modelo de bigramas: 0.005242914579016099
Modelo de trigramas: 1.940880771694195e-05
Modelo de tetragramas: 0.020976445760095178


Para los ejemplos, a propósito escogí secuencias de palabras con mucha frecuencia en el corpus, para corroborar que les asignara una probabilidad significativa. La oración "conferencia de prensa matutina" se encuentra en todos los archivos de las conferencias, por lo que es razonable que $P_3$["matutina" | "conferencia", "de", "prensa"] nos haya dado un número significativo (0.2). Además se observa que el suavizado permite asignar probabilidades distintas de 0 a secuencias no vistas. Sin suavizado, la probabilidad condicional de "capibara", dado cualquier contexto, sería 0.

---

### 3.3- Construya un modelo con interpolación

Construya un modelo interpolado con valores $\lambda$ fijos:
$$
\hat{P}(w_n | w_{n-3}w_{n-2}w_{n-1}) = \lambda_1 P(w_n | w_{n-3}w_{n-2}w_{n-1}) + \lambda_2 P(w_n | w_{n-2}w_{n-1}) + \lambda_3 P(w_n | w_{n-1}) + \lambda_4  P(w_n).
$$
Para ello experimente con el modelo con alguna partición, por ejemplo, de 80%, 10% y 10% para entrenar (train), ajuste de parámetros (val) y prueba (test) respectivamente. Muestre como bajan o suben para algunas pruebas las perplejidades (implementa tu perplejidad) en validación, finalmente pruebe SOLO una vez en test. Para esto puede explora manualmente 3 conjuntos distintos de valores $\vec{\lambda}$ y elija el mejor.

---

La siguiente clase implementa la lógica para un modelo interpolado. Recibe un número $n$ para determinar que se usarán los modelos de $i$-gramas para $i=1, ..., n$. Recibe la lista de coeficientes de cada modelo, el i-ésimo coeficiente corresponde al modelo de $(i+1)$-gramas. También recibe el $k$ para la estrategia Add-$k$ de cada modelo.

In [42]:
class InterpolatedModel:
    def __init__(self, n: int, coeff: list[float], text_processor: TextProcessor, k: float = 1.0):
        # coeff is the array of coefficientes assigned to each n-gram model. the i-th coefficient correspond to the (i+1)-gram model

        if len(coeff) != n:
            raise ValueError("coeff should have length n")

        self.n = n
        self.coeff = coeff
        self.k = k
        self.text_processor = text_processor
        self.models = [NgramLanguageModel(n=i+1, text_processor=processor, k=k) for i in range(self.n)]

    def fit(self, corpus: list[list[str]]):
        for i in range(self.n):
            self.models[i].fit(corpus)

        return

    def conditional_prob(self, word: str, context: list[str]) -> float:
        if len(context) != self.n - 1:
            raise ValueError("context should be of length n-1")

        word = self.text_processor.mask_oov(word)
        context = self.text_processor.mask_oov(context)

        prob = 0
        for i in range(self.n):
            p = self.models[i].conditional_prob(word, context[self.n - 1 - i:])
            prob += self.coeff[i]*p

        return prob

    def prob_sequence(self, sequence: list[str], log_prob: bool = False) -> float:
        # check if sequence has <s> token at the beggining, and if so add the necessary <s> tokens to compute correctly the probability
        if self.n >= 3 and sequence[0] == self.text_processor.INIT_TKN:
            sequence = [self.text_processor.INIT_TKN]*(self.n-2) + sequence

        if len(sequence) < self.n:
            raise ValueError("sequence should be at least of length n, except in the case where sequence starts with <s> (only one). In the latter case, sequence should be at least of length 2")

        sequence = list(map(self.text_processor.mask_oov, sequence))

        # compute the chain rule for the probability of the sequence (using logarithm)
        log_p = 0
        for i in range(self.n-1, len(sequence)):
            # when len(doc) - n + 1 <= 0, it does not enter to the for loop. This makes sense because that means doc does not have n-grams
            n_gram = sequence[i-self.n+1 : i+1]  # n-gram
            log_p += math.log(self.conditional_prob(n_gram[-1], n_gram[:-1]))

        if log_prob:
            return log_p

        return math.exp(log_p)

La siguiente función calcula la perplejidad de un corpus dado un modelo interpolado.

Hay que tener cuidado con la perplejidad porque cuando el logaritmo de la probabilidad de una secuencia de tokens es menor a -750, entonces le asignaríamos una probabilidad de 0 a la secuencia. Esto es debido a que ``math.exp(x)`` da 0 por el error numérico, para x<-750. Para ilustrar esto, consideremos un texto de 200 tokens. Podemos calcular la probabilidad de la secuencia, usando la regla de la cadena, como un producto de probabilidades condicionales. Si suponemos que cada probabilidad condicional que aparece en el producto tiene un valor promedio de ``math.exp(-5)`` (``0.00673``, lo cual es razonable), entonces la log-probabilidad de la secuencia se puede estimar como ``(log probabilidad condicional) * (número de tokens)= -5*(200)=-1000``. Asignando así un 0 a la probabilidad de la secuencia y la perplejidad quedaría indefinida.

Para evitar este problema, trabajamos con log-probabilidades.

In [78]:
def perplexity(model: InterpolatedModel, corpus: list[list[str]]) -> float:
    corpus_len = sum(len(text) for text in corpus)
    log_p = 0

    for i, text in enumerate(corpus):
        log_p += model.prob_sequence(text, log_prob=True)

    return math.exp(-log_p/corpus_len)

Dividimos el corpus en conjunto de entrenamiento (70%), valicación (15%) y prueba (15%).

In [44]:
corpus_train, corpus_val = train_test_split(corpus_masked, train_size=0.8, random_state=42)
corpus_val, corpus_test = train_test_split(corpus_val, train_size=0.5, random_state=7)

Exploración de coeficientes

In [45]:
n = 4
coeff1 = [0.25, 0.25, 0.25, 0.25]
coeff2 = [0.1, 0.2, 0.3, 0.4]  # Increasing
coeff3 = [0.4, 0.3, 0.2, 0.1]  # Decreasing

In [46]:
interpolated_model = InterpolatedModel(n=n, coeff=coeff1, text_processor=processor, k=0.1)
interpolated_model.fit(corpus_train)

Perplejidad de modelo interpolado, 1er conjunto de coeficientes.

In [79]:
perplexity(interpolated_model, corpus_val)

163.39447878038524

Perplejidad de modelo interpolado, 2do conjunto de coeficientes.

In [80]:
interpolated_model.coeff = coeff2
perplexity(interpolated_model, corpus_val)

200.71983252025345

Perplejidad de modelo interpolado, 3er conjunto de coeficientes.

In [81]:
interpolated_model.coeff = coeff3
perplexity(interpolated_model, corpus_val)

145.84137417870633

El mejor conjunto de coeficientes fue ``[0.4, 0.3, 0.2, 0.1]``. La perplejidad sobre el conjunto de prueba usando estos coeficientes es la siguiente:

In [82]:
perplexity(interpolated_model, corpus_test)

145.50024803092217

El mejor conjunto de coeficientes favorece a los modelos de unigramas y bigramas. Esto confirma lo que pensaba Bengio en su artículo *A Neural Probabilistic Language Model* (2003) acerca de que los modelos de $n$-gramas dependian principalmente de los unigramas, bigramas y trigramas. Por lo que estos modelos estadísticos no logran *modelar el lenguaje* bien debido a su incapacidad de entender un contexto más grande.

---

## 4- Generación de Texto

Para esta parte reentrenará su modelo de lenguaje interpolado para aprender los valores $\lambda$:
$$
\hat{P}(w_n | w_{n-3}w_{n-2}w_{n-1}) = \lambda_1 P(w_n | w_{n-3}w_{n-2}w_{n-1}) + \lambda_2 P(w_n | w_{n-2}w_{n-1}) + \lambda_3 P(w_n | w_{n-1}) + \lambda_4  P(w_n).
$$


### 4.1- Estrategia con base en *Expectation Maximization*

Proponga una estrategia con base en Expectation Maximization (investigue por su cuenta sobre EM) para encontrar buenos valores de interpolación en $\hat{P}$ usando todo el dataset (se adjunta un material de apoyo de Jacob Eisenstein). Para ello experimente con el modelo en particiones, por ejemplo, de 80%, 10% y 10% para entrenar (train), ajustar parámetros (val) y probar (test) respectivamente. Muestre como bajan las perplejidades en 5 iteraciones que usted elija (de todas las que sean necesarias de acuerdo a su EM) en validación, y pruebe una vez en test. Sino logra hacer este punto, haga todos los siguientes puntos con el modelo de lenguaje con algunos $\lambda$ fijados manualmente.


---

A continuación se implementa el algoritmo E-M.

In [71]:
def expectation_maximization(
    corpus: list[list[str]],
    model: InterpolatedModel,
    n_iters: int,
    history: Optional[list[float]] = None
) -> list[float]:

    lambdas = np.array([1/model.n]*model.n)

    if history is not None:
        history.append(lambdas)

    for _ in range(n_iters):
        # E-step: Compute expected counts
        weights = np.zeros(model.n)

        for text in corpus:
            # check if sequence has <s> token at the beggining, and if so add the necessary <s> tokens to compute correctly the probability
            if model.n >= 3 and text[0] == model.text_processor.INIT_TKN:
                text = [model.text_processor.INIT_TKN] * (model.n - 2) + text

            if len(text) < model.n:
                raise ValueError(
                    "Each text should be at least of length n, except in the case where sequence starts with <s> (only one). In the latter case, text should be at least of length 2"
                )

            text = list(map(model.text_processor.mask_oov, text))

            for i in range(model.n - 1, len(text)):
                # when len(doc) - n + 1 <= 0, it does not enter to the for loop. This makes sense because that means doc does not have n-grams
                n_gram = text[i - model.n + 1 : i + 1]  # n-gram

                word = n_gram[-1]
                context = n_gram[:-1]

                probs = np.array([model.models[j].conditional_prob(word, context[model.n - 1 - j :]) for j in range(model.n)])
                total_p = np.sum(probs)
                probs /= total_p

                weights += probs

        # M-step: Normalize new lambda values
        total_weight = weights.sum()
        lambdas = weights/total_weight

        if history is not None:
            history.append(lambdas)

    model.coeff = lambdas # update interpolated model coefficients

    return lambdas

Entrenamos los coeficientes $\lambda$'s con E-M durante 10 iteraciones, utilizando el conjunto de validación, y mostramos los resultados.

In [72]:
lambdas_history = []
lambdas = expectation_maximization(corpus_val, interpolated_model, n_iters=10, history=lambdas_history)

In [73]:
lambdas_history

[array([0.25, 0.25, 0.25, 0.25]),
 array([0.28502087, 0.40860887, 0.2124263 , 0.09394396]),
 array([0.28502087, 0.40860887, 0.2124263 , 0.09394396]),
 array([0.28502087, 0.40860887, 0.2124263 , 0.09394396]),
 array([0.28502087, 0.40860887, 0.2124263 , 0.09394396]),
 array([0.28502087, 0.40860887, 0.2124263 , 0.09394396]),
 array([0.28502087, 0.40860887, 0.2124263 , 0.09394396]),
 array([0.28502087, 0.40860887, 0.2124263 , 0.09394396]),
 array([0.28502087, 0.40860887, 0.2124263 , 0.09394396]),
 array([0.28502087, 0.40860887, 0.2124263 , 0.09394396]),
 array([0.28502087, 0.40860887, 0.2124263 , 0.09394396])]

In [74]:
interpolated_model.coeff

array([0.28502087, 0.40860887, 0.2124263 , 0.09394396])

A continuación se muestran las log-perplejidades para las primeras 5 iteraciones.

In [83]:
for i in range(5):
    print(f"Iteración {i}, Coeficientes {lambdas_history[i]}")
    interpolated_model.coeff = lambdas_history[i]
    print("Perplejidad en conjunto de validación:", perplexity(interpolated_model, corpus_val), "\n")

Iteración 0, Coeficientes [0.25 0.25 0.25 0.25]
Perplejidad en conjunto de validación: 163.39447878038524 

Iteración 1, Coeficientes [0.28502087 0.40860887 0.2124263  0.09394396]
Perplejidad en conjunto de validación: 138.74684294224096 

Iteración 2, Coeficientes [0.28502087 0.40860887 0.2124263  0.09394396]
Perplejidad en conjunto de validación: 138.74684294224096 

Iteración 3, Coeficientes [0.28502087 0.40860887 0.2124263  0.09394396]
Perplejidad en conjunto de validación: 138.74684294224096 

Iteración 4, Coeficientes [0.28502087 0.40860887 0.2124263  0.09394396]
Perplejidad en conjunto de validación: 138.74684294224096 



La perplejidad del conjunto de prueba utilizando los últimos coeficientes es:

In [84]:
interpolated_model.coeff = lambdas_history[-1]
perplexity(interpolated_model, corpus_val)

138.74684294224096

Se puede ver cómo desde la primera iteración se obtuvo una convergencia. Además, también vimos que se minimizó la perplejidad un poco más comparando con los mejores coeficientes que se exploraron de manera manual en el ejercicio 3.3:

- Coeficientes fijos ``[0.4, 0.3, 0.2, 0.1]`` -> perplejidad sobre el conjunto de prueba: 145.50
- Coeficientes con EM ``[0.28502087, 0.40860887, 0.2124263 , 0.09394396]]`` -> perplejidad sobre el conjunto de prueba: 138.74

En los coeficientes encontrados con EM vemos cómo el mayor peso de 0.4 se le asigna al modelo de bigramas, después al modelo de unigramas y trigramas con pesos similares de 0.2, y al modelo de tetragramas se le da un peso muy bajo de 0.09. De nuevo confirmando lo que decíamos que los modelos basados en n-gramas se basaban principalmente en unigramas, bigramas y trigramas.

---

### 4.2- Función para generar texto

Haga una función "generar texto" con base en su modelo de lenguaje $\hat{P}$ del último punto. El modelo deberá poder parar automáticamente cuando genere el símbolo de terminación de oración al final (e.g., ``</s>``), o 50 palabras. Proponga algo para que en los últimos tokens sea más probable generar el token ``</s>``. Muestre al menos cinco ejemplos.

---

**EJERCICIO CON AYUDA DE LESLIE JANETH**

Para generar el siguiente token, dado un contexto, calculamos las probabilidades condicionales de todo el vocabulario (excepto el token de inicio) y sampleamos una palabra de los top k tokens con mayor probabilidad y el token ``</s>`` (o sea, k+1 tokens en total para el sampling).

La estrategia para aumentar la probabilidad del token ``</s>`` en los últimos tokens me la comentó **Leslie Quincosa**, y vi su código para darme una idea de cómo integrarla a mi código. Se trata de que después de una cierta iteración ``boost_threshold`` se multiplique la probabilidad del token ``</s>`` por un cierto factor ``boost_factor``.

Antes de samplear, se deben normalizar las probabilidades ya que no sumarán 1 pues quitamos el token inicial ``<s>`` y porque en las últimas iteraciones también modificamos la probabilidad de ``</s>``.

La siguiente función puede recibir una lista de tokens iniciales de tamaño n-1, donde $n$ es el tamaño de $n$-gramas más grande que se utiliza en un modelo interpolado (en nuestro caso $n=4$). Si no se manda la lista de tokens iniciales, se agregan n-1 tokens iniciales ``<s>``.

In [105]:
def generate_text(
    model: InterpolatedModel,
    n_tokens: int = 50,
    top_k : int = 1000,
    boost_threshold: int = 45,
    boost_fastor: float = 5,
    seed: Optional[list[str]] = None
) -> str:
    if seed is None:
        text = [model.text_processor.INIT_TKN] * (model.n - 1)
    else:
        if len(seed) != model.n - 1:
            raise ValueError("seed should be of length n-1")

        text = list(map(model.text_processor.mask_oov, seed))

    vocab = list(
        model.text_processor.vocab - {model.text_processor.INIT_TKN}
    )  # we quit the initial token

    for i in range(n_tokens):
        context = text[-model.n + 1 :]  # last n-1 tokens from generated text

        probs = [
            (w, model.conditional_prob(w, context))
            for w in vocab
            if w != model.text_processor.END_TKN
        ]

        probs.sort(key=lambda x: x[1], reverse=True)

        words = [w for w, p in probs[:top_k]]
        probs = [p for w, p in probs[:top_k]]

        prob_end_tkn = model.conditional_prob(
            model.text_processor.END_TKN, context
        )

        if i + 1 >= boost_threshold:
            # increase </s> probability
            prob_end_tkn *= boost_fastor

        probs.append(prob_end_tkn)
        words.append(model.text_processor.END_TKN)

        # normalize probabilities, since we quit <s> token and they will not sum 1 (and also we may altered </s> probability)
        total_prob = sum(probs)
        probs = [p / total_prob for p in probs]

        # we sample the next word according to the conditional probabilities
        next_word = np.random.choice(words, p=probs).item() # .item() to get raw string (not numpy string)
        text.append(next_word)

        if next_word == model.text_processor.END_TKN:
            break

    return text

5 ejemplos, sampleando sobre los top 1000 (valor default de la función) tokens de mayor probabilidad:

In [108]:
np.random.seed(10)
print(" ".join(generate_text(interpolated_model)))

<s> <s> <s> mismo <unk> van del gabinete <unk> en tamaulipas <unk> que ellos total de cinco hacer eso sería mi sobrina unidos a <unk> por no el más al principio muy rápido estaba como datos que <unk> la secretaría pero <unk> va a beneficiar a enfrentar que cuidar a ciudadanos que no


In [109]:
np.random.seed(7)
print(" ".join(generate_text(interpolated_model)))

<s> <s> <s> <unk> etcétera <unk> programa denuncia los cuales <unk> el agua informa cosas que se puede debatir de no opino <unk> donde claudia sheinbaum <unk> </s>


In [110]:
np.random.seed(11)
print(" ".join(generate_text(interpolated_model)))

<s> <s> <s> de la gasolina eso se nos <unk> una hacienda estoy ahí <unk> </s>


In [111]:
np.random.seed(13)
print(" ".join(generate_text(interpolated_model)))

<s> <s> <s> otro tema pendiente periódico </s>


In [112]:
np.random.seed(17)
print(" ".join(generate_text(interpolated_model)))

<s> <s> <s> la vez que se planteó yo siempre por ciento de influencia <unk> </s>


Se observa que en general no tiene sentido el texto generado, salvo en ciertas frases de longitud 3 o 4, por ejemplo: "hacer eso sería", "donde claudia sheinbaum" y "otro tema pendiente". Por cierto, tomar sólo los top-k tokens para samplear fue una mejor estrategia comparado con samplear sobre todo el vocabulario. Intenté al principio samplear sobre todo el vocabulario y el modelo no generaba cosas con sentido. Después intenté con varios valores de k, y encontré que 1000 era un buen número.

---

### 4.3- Función para generar texto con semilla

Haga una función "generar texto con semilla", que reciba tres tokens y a partir de ellos empiece a generar texto automáticamente con la misma política del punto anterior.


---

La función anterior ya tiene la opción para recibir una semilla.

In [113]:
np.random.seed(10)
print(" ".join(generate_text(interpolated_model, seed = ["buenos", "días", "cómo"])))

buenos días cómo enfrentar <unk> tienen tiempo se les da yo no <unk> ustedes llegaron <unk> ‘ necesito son sólo el haya ’ y de una medida a tener problemas para porque por otros es fueron la de méxico <unk> </s>


In [114]:
np.random.seed(7)
print(" ".join(generate_text(interpolated_model, seed = ["presidente", "andrés", "manuel"])))

presidente andrés manuel lópez de salud por semanas más grande <unk> la nación ’ me dice <unk> el efecto <unk> hay trabajos de como dé de estas adquisiciones <unk> son le podría por la venta del consejo coordinador pueden nacional de ellos no tienen de otros tiempos del fraude la sería el cómo


In [115]:
np.random.seed(11)
print(" ".join(generate_text(interpolated_model, seed = ["la", "educación", "es"])))

la educación es que se nos informa a haber <unk> hay señal temas le voy arriba centavos <unk> ayer <unk> que el pueblo una ser una serie de la instrucción por ciento de el donde eso fue se continúe </s>


In [119]:
np.random.seed(23)
print(" ".join(generate_text(interpolated_model, seed = ["la", "mafia", "del"])))

la mafia del imss vaya era la fiscalía los estados <unk> usted sabe <unk> </s>


In [117]:
np.random.seed(17)
print(" ".join(generate_text(interpolated_model, seed = ["bola", "de", "corruptos"])))

bola de corruptos de cómo <unk> <unk> kilogramos todo se porque <unk> por cuanto a usar precios <unk> antes y ahí y el poder público de repente un la la <unk> saben que no yo también <unk> muy importante lo que están libres ejemplo <unk> </s>


Encontramos el mismo comportamiento que en el ejercicio pasado: no hay coherencia a nivel global pero sí se generan ciertas frases de tamaño 3 o 4 con cierto sentido. Por ejemplo, después de la semilla "presidente andrés manuel" el modelo predijo "lópez" como la siguiente palabra.

---

### 4.4- Top 5 de permutaciones más y menos probables

Haga una función que reciba una oración, y que permute todos sus tokens, que los evalué todos con su modelo de lenguaje, y que muestre el top 5 más probable y el top 5 menos probable, por ejemplo, use las siguientes dos oraciones y una más que usted proponga: "sino gano me voy a la chingada", "ya se va a acabar la corrupción".

---

In [126]:
def top_5_permutations(model: InterpolatedModel, sentence: list[str]):
    permuts = list(permutations(sentence))
    probs = [(i, model.prob_sequence(perm)) for i, perm in enumerate(permuts)]
    probs.sort(key=lambda x: x[1], reverse=True)

    print("5 permutaciones más probables")
    for i in range(5):
        print(f"Permutación: {permuts[probs[i][0]]} - probabilidad {probs[i][1]}")

    print("\n5 permutaciones menos probables")
    for i in range(5):
        print(f"Permutación: {permuts[probs[-i-1][0]]} - probabilidad {probs[-i-1][1]}")

Ejemplos:

In [127]:
top_5_permutations(interpolated_model, ["sino", "gano", "me", "voy", "a", "la", "chingada"])

5 permutaciones más probables
Permutación: ('gano', 'chingada', 'sino', 'me', 'voy', 'a', 'la') - probabilidad 1.7495065427588989e-07
Permutación: ('chingada', 'gano', 'sino', 'me', 'voy', 'a', 'la') - probabilidad 1.7495065427588989e-07
Permutación: ('sino', 'gano', 'chingada', 'me', 'voy', 'a', 'la') - probabilidad 9.771204919810925e-08
Permutación: ('gano', 'sino', 'chingada', 'me', 'voy', 'a', 'la') - probabilidad 9.771204919810925e-08
Permutación: ('sino', 'chingada', 'gano', 'me', 'voy', 'a', 'la') - probabilidad 9.7694556163849e-08

5 permutaciones menos probables
Permutación: ('me', 'a', 'la', 'gano', 'sino', 'voy', 'chingada') - probabilidad 7.242773167347593e-19
Permutación: ('me', 'a', 'la', 'gano', 'voy', 'chingada', 'sino') - probabilidad 7.433530787830064e-19
Permutación: ('me', 'a', 'la', 'gano', 'voy', 'sino', 'chingada') - probabilidad 7.716713247758474e-19
Permutación: ('me', 'a', 'la', 'gano', 'sino', 'chingada', 'voy') - probabilidad 8.054897817104227e-19
Permutació

In [128]:
top_5_permutations(interpolated_model, ["ya", "se", "va", "a", "acabar", "la", "corrupción"])

5 permutaciones más probables
Permutación: ('corrupción', 'acabar', 'ya', 'se', 'va', 'a', 'la') - probabilidad 2.1258048231672103e-05
Permutación: ('acabar', 'corrupción', 'ya', 'se', 'va', 'a', 'la') - probabilidad 2.1258044742922328e-05
Permutación: ('acabar', 'ya', 'se', 'va', 'a', 'la', 'corrupción') - probabilidad 3.4207132717377285e-06
Permutación: ('ya', 'acabar', 'se', 'va', 'a', 'la', 'corrupción') - probabilidad 2.8496249574912944e-06
Permutación: ('ya', 'acabar', 'corrupción', 'se', 'va', 'a', 'la') - probabilidad 2.7525522682623987e-06

5 permutaciones menos probables
Permutación: ('se', 'a', 'la', 'acabar', 'va', 'ya', 'corrupción') - probabilidad 2.5658971917728768e-15
Permutación: ('se', 'a', 'la', 'acabar', 'va', 'corrupción', 'ya') - probabilidad 2.737725994629387e-15
Permutación: ('a', 'se', 'la', 'acabar', 'va', 'ya', 'corrupción') - probabilidad 3.1119838374297877e-15
Permutación: ('a', 'la', 'se', 'acabar', 'va', 'ya', 'corrupción') - probabilidad 3.18969150626158

In [129]:
top_5_permutations(interpolated_model, ["presidente", "de", "los", "estados", "unidos"])

5 permutaciones más probables
Permutación: ('presidente', 'los', 'estados', 'unidos', 'de') - probabilidad 0.005188333903081463
Permutación: ('presidente', 'de', 'los', 'estados', 'unidos') - probabilidad 0.004510253939202986
Permutación: ('los', 'presidente', 'estados', 'unidos', 'de') - probabilidad 0.004387904439836313
Permutación: ('de', 'presidente', 'los', 'estados', 'unidos') - probabilidad 0.002149065859174432
Permutación: ('los', 'presidente', 'de', 'estados', 'unidos') - probabilidad 0.0017581272819283706

5 permutaciones menos probables
Permutación: ('presidente', 'de', 'los', 'unidos', 'estados') - probabilidad 6.585041855677052e-08
Permutación: ('los', 'presidente', 'de', 'unidos', 'estados') - probabilidad 6.701678219201393e-08
Permutación: ('de', 'presidente', 'los', 'unidos', 'estados') - probabilidad 6.70616750774683e-08
Permutación: ('presidente', 'los', 'de', 'unidos', 'estados') - probabilidad 6.709000345389876e-08
Permutación: ('los', 'de', 'presidente', 'unidos', 

En los ejemplos se puede ver que las permutaciones que salen en el top 5 contienen frases con coherencia, y en las peores permutaciones no hay ninguna frase con coherencia. Por ejemplo, en la primera oración las permutaciones con mayor probabilidad contienen la frase  "me voy a la", lo cual es una frase con coherencia y por eso se le asigna una probabilidad alta. En la segunda oración las permutaciones top continenen la frase "ya se va a la", y en la tercera oración tienen la frase "los estados unidos".

---

### 4.5- Top de 5 palabras más probables después de tres dadas.

Haga una función que reciba tres palabras y regrese las siguientes 5 palabras más probables dadas las tres primeras. Muéstrelas en pantalla.

---

In [130]:
def top_5_word_after_context(model: InterpolatedModel, context: list[str]):
    if len(context) != model.n - 1:
        raise ValueError("context should be of length n-1")

    context = list(map(model.text_processor.mask_oov, context))

    probs = [(w, model.conditional_prob(w, context)) for w in model.text_processor.vocab]
    probs.sort(key=lambda x: x[1], reverse=True)

    print(f"Top 5 palabras más probables después de {context}")
    for i in range(5):
        print(f"Palabra: {probs[i][0]} - probabilidad {probs[i][1]}")


Ejemplos:

In [132]:
top_5_word_after_context(interpolated_model, ["la", "educación", "es"])

Top 5 palabras más probables después de ['la', 'educación', 'es']
Palabra: <unk> - probabilidad 0.048320691556252085
Palabra: el - probabilidad 0.043479418196520556
Palabra: que - probabilidad 0.04150031121550378
Palabra: un - probabilidad 0.04112206623231236
Palabra: la - probabilidad 0.038186814545824954


In [133]:
top_5_word_after_context(interpolated_model, ["la", "mafia", "del"])

Top 5 palabras más probables después de ['la', 'mafia', 'del']
Palabra: <unk> - probabilidad 0.039022922159146226
Palabra: gobierno - probabilidad 0.016518917026294436
Palabra: estado - probabilidad 0.016320043733837362
Palabra: poder - probabilidad 0.014137318295140372
Palabra: país - probabilidad 0.013835593170272286


In [139]:
top_5_word_after_context(interpolated_model, ["andrés", "manuel", "lópez"])

Top 5 palabras más probables después de ['andrés', 'manuel', 'lópez']
Palabra: obrador - probabilidad 0.6301703440576965
Palabra: <unk> - probabilidad 0.04005505401584125
Palabra: de - probabilidad 0.012818338957356552
Palabra: </s> - probabilidad 0.010104632108758586
Palabra: <s> - probabilidad 0.010074783551905853


Con estos ejemplos vemos que el top 5 de palabras después de un contexto dado (de longitud 3) hace sentido. Por ejemplo, el modelo le asigna una probabilidad alta a "poder" después de "la mafia del", y también a la palabra "obrador" después de "andrés manuel lópez".

---

## 5- El Ahorcado

Para esta parte estudie y comprenda el funcionamiento de la estrategia propuesta por [Norvig](http://norvig.com/spell-correct.html). Siéntete libre de adaptar y/o extender parcial o totalmente el código de Norvig para esta tarea.
Diseñe una función que sea capaz de encontrar los caracteres faltantes de una palabra. Para ello proponga una adaptación simple de la estrategia de corrección ortográfica propuesta por Norvig. La función de el ahorcado debe poder tratar con hasta 4 caracteres desconocidos en palabras de longitud arbitraria. La función debe trabajar en tiempo razonable (≈ 1 minuto en una laptop o menos). La función debe trabajar como sigue con 10 ejemplos:
~~~
>>> hangman("pe_p_e")
"people"
>>> hangman("phi__sop_y")
"philosophy"
>>> hangman(" si_nif_c_nc_ ")
"significance"
~~~
Puede resolver este punto con una extensión MUY simple (hasta la MÁS OBVIA) de la estrategia de Norvig, PERO HAY FORMAS MUCHO MÁS EFICIENTES (Si te sobra tiempo en la vida) con distancias de edición (e.g., Levenshtein) o de subcadenas (e.g., Karp Rabin, Aho-Corasick, Tries, etc.).

Comente brevemente como integraría un modelo de lenguaje con el modelo de Norvig para tratar de resolver errores gramaticales de más alto nivel, o errores dónde el error sea una palabra que SÍ está en el diccionario, por ejemplo: *"In the science off Maths ..."*.

---



Para crear la función del ahorcado (``hangman``), vamos a utilizar expresion regulares (regex). En regex, el caracter "." sirve para hacer match a cualquier caracter. De manera que si reemplazamos los "_" por "." en una expresión regular, podemos encontrar todas las palabras en el vocabulario que tengan el patrón que recibe el ahorcado. A la expresión regular también le debemos agregar los caracteres de inicio ("^") y final de string ("$"). Después con nuestro modelo de unigramas previamente entrenado, podemos obtener la probabilidad de cada palabra que hizo match a la expresión regular, y la respuesta sería la de mayor probabilidad.

Esta estrategia se inspira en la idea de Norvig ya que él busca palabras candidatas considerando los errores más comúnes: inserciones, eliminaciones, sustitucioenes y transposiciones de caracteres. Y después esas palabras las ordena de acuerdo a su probabilidad de unigrama (sin suavizado) en un corpus. Nosotros en vez de considerar los posibles errores, simplemente buscamos las palabras candidatas mediante la expresión regular que define el patrón del ahorcado (con la estrategia explicada en el párrafo pasado).

In [164]:
def hangman(pattern: str) -> str:

    # create regex pattern, replacing "_" by "." and adding start ("^") and end ("$") of string characters
    regex = "^" + pattern.replace("_", ".") + "$"
    # compile regex
    regex = re.compile(regex)

    # find matching words in vocabulary. regex.match(word) returns None if no matching
    candidates = [word for word in unigram_model.text_processor.vocab if regex.match(word)]

    # unigram probabilities
    probs = [unigram_model.conditional_prob(word, []) for word in candidates]

    max = 0
    max_id = -1

    for i in range(len(probs)):
        if probs[i] > max:
            max = probs[i]
            max_id = i

    return candidates[max_id]

10 ejemplos:

In [167]:
print(hangman("pr_s_de_te"))
print(hangman("m_f_a"))
print(hangman("cor_up_ió_"))
print(hangman("an___s"))
print(hangman("sh__nba_m"))
print(hangman("e_u_a_i__"))
print(hangman("mat_t_n_"))
print(hangman("e_t_d_s"))
print(hangman("ust_d__"))
print(hangman("h_l_"))

presidente
mafia
corrupción
andrés
sheinbaum
educación
matutina
estados
ustedes
hola


Aquí en los ejemplos vemos que sí se está utilizando la probabilidad para determinar la respuesta. Por ejemplo, el patrón "an___s" tiene varias palabras posibles, algunas de ellas son: "anchas", "andrés" y "ansias". Pero la de mayor probabilidad, como esperamos, es "andrés":

In [170]:
test_words = ["anchas", "andrés", "ansias"]
for word in test_words:
    print("Palabra:", word)
    print("En vocabulario:", word in unigram_model.text_processor.vocab)
    print("Probabilidad:", unigram_model.conditional_prob(word, []), "\n")

Palabra: anchas
En vocabulario: True
Probabilidad: 1.8128173926249346e-06 

Palabra: andrés
En vocabulario: True
Probabilidad: 0.0025036911412510054 

Palabra: ansias
En vocabulario: True
Probabilidad: 8.612072127693258e-07 



Lo que se me ocurre para resolver errores en una oración es lo siguiente:

1.   De la oración, suponer que cada palabra puede estar mal escrita.
2.   Por cada palabra, intercambiar esa palabra por otra en el vocabulario y calcular la probabilidad de la oración con un modelo de lenguaje.
3.   Así creamos una lista de oraciones que se obtiene al sustituir una palabra por otra en el vocabulario, y sus respectivas probabilidades.
4.   Tomamos la oración con mayor probabilidad.

Pero esta idea tiene un problema y es que no hay manera de determinar si la oración con mayor probabilidad tiene las palabras que se estaban buscando. Por ejemplo, en la oración "In the science off Maths" podemos obtener las siguientes alternativas:

*   “In the science of Maths”
*   "In the science with Maths"

La primera oración es la que nos gustaría, pero la segunda oración podría ser resultado de este procedimiento. Una manera de corregir este problema es que en vez de sustituir una palabra de la oración por cualquier otra en el vocabulario, podemos tomar la idea de Norvig de considerar sólo candidatos con los errores más comúnes de escritura. Pero de todos modos el pipeline completo pienso que es costoso.



